In [1]:
import pandas as pd
import os

# Trabalhando com as partições

Depois de rodar o script `etl.py`, os dataframes ficarão salvos na pasta out_parquet. Basicamente, eles são particionados com base no `chunksize` especificado.
A partir disso, você tem algumas opções para ler essas partições. Optei por carregar e fazer operações dentro de um loop para cada chunk.
Como exemplo, vamos fazer o mesmo filtro utilizado no `etl_dados.ipynb` para encontrar apenas pessoas com 60 anos ou mais, e junta-las em um dataframe no final.

In [2]:
partes = []
for arquivo in sorted(os.listdir("out_parquet")):
    df = pd.read_parquet(f"out_parquet/{arquivo}")
    if "C008" in df.columns: # C008 é o código da variável de idade
        df["C008"] = pd.to_numeric(df["C008"], errors="coerce")
        idosos = df[df["C008"] >= 60]
        print(f"Total de idosos: {len(idosos)}")
        partes.append(idosos)

idosos = pd.concat(partes, ignore_index=True)
print("Final:", idosos.shape)
print(idosos["C008"].head(5))

Total de idosos: 5385
Total de idosos: 6509
Total de idosos: 7652
Total de idosos: 8233
Total de idosos: 8940
Total de idosos: 6835
Final: (43554, 1088)
0    69.0
1    78.0
2    81.0
3    81.0
4    72.0
Name: C008, dtype: float64


Agrupar todos na memória como fiz não é exatamente eficiente, mas servirá já que não estamos trabalhando com tantas linhas de dado.
Vamos importar o dicionário também para auxiliar nas variáveis:

In [3]:
from dic_base import DicBase

dic = DicBase()
dic.describe_var("C008")


 C008 — Idade do morador na data de referência
Valores possíveis:
  000 a 130 → Idade (em anos)
  <NA> → Não aplicável


Vamos agora continuar aplicando filtros. A forma como fiz na parte dos idosos foi com:
```python
idosos = df[df["C008"] >= 60]
```
Que é indexação booleana. Ela é muito poderosa, e pode ser utilizada para aplicar vários filtros em seguida. Por exemplo, vamos filtrar todos os homens que sabem ler e escrever.

In [4]:
dic.describe_var("C006")
dic.describe_var("D001")


 C006 — Sexo
Valores possíveis:
    1 → Homem
    2 → Mulher
  <NA> → Não aplicável

 D001 — Sabe ler e escrever
Valores possíveis:
    1 → Sim
    2 → Não
    9 → Ignorado
  <NA> → Não aplicável


##

In [5]:
idosos["C006"] = pd.to_numeric(idosos["C006"], errors="coerce")
idosos["D001"] = pd.to_numeric(idosos["D001"], errors="coerce")
idosos_aux = idosos[(idosos["C006"] == 1) & (idosos["D001"] == 1)]
print(idosos_aux.shape)

(14527, 1088)


Você pode ter o mesmo resultando comparando assim:

In [6]:
cond_homem = idosos["C006"] == 1
cond_mulher = idosos["C006"] == 2
cond_ler = idosos["D001"] == 1

homens_leitores = idosos[cond_ler & cond_homem]
mulheres_leitoras = idosos[cond_ler & cond_mulher]

print("idosos leitores", homens_leitores.shape)
print("idosas leitoras", mulheres_leitoras.shape)

idosos leitores (14527, 1088)
idosas leitoras (18667, 1088)


Para auxiliar nas análises, temos outra classe utilitária:

In [7]:
cod_domicilio = "A001"
dic.describe_var(cod_domicilio)
cod_rede_agua = "A005012"
dic.describe_var(cod_rede_agua)
cod_dia_nascimento = "C00701"
dic.describe_var(cod_dia_nascimento)


 A001 — Tipo do domicílio
Valores possíveis:
    1 → Casa
    2 → Apartamento
    3 → Habitação em casa de cômodos, cortiço ou cabeça de porco
    9 → Ignorado
  <NA> → Não aplicável

 A005012 — Este domicílio está ligado à rede geral de distribuição de água?
Valores possíveis:
    1 → Sim
    2 → Não
    9 → Ignorado
  <NA> → Não aplicável

 C00701 — Dia de nascimento
Valores possíveis:
  01 a 31 → Dia de nascimento
   99 → Não informado
  <NA> → Não aplicável


In [8]:
from pd_utils import between_numeric, compare_numeric, is_na

# Seguindo a ideia de criar as condições primeiro
cond_casa = compare_numeric(idosos, cod_domicilio, "==", 1)
cond_agua = compare_numeric(idosos, cod_rede_agua, "==", 1)

selecionados = idosos[cond_casa & cond_agua]
print("Água e casa:", selecionados.shape)

# Ambos ao mesmo tempo
selecionados = idosos[
    compare_numeric(idosos, cod_domicilio, "==", 2) &
    compare_numeric(idosos, cod_rede_agua, "==", 1)
]
print("Água e apartamento:", selecionados.shape)

# Sem respostas (Comparar usando o is_na)
selecionados = idosos[
    is_na(idosos, cod_rede_agua)
]
print("não aplicavel (água):", selecionados.shape)

# Range de valores
selecionados = idosos[
    between_numeric(idosos, cod_dia_nascimento, 1, 15) # Entre o dia 1 e 15 (inclusive como default = both)
]
print("idosos que nasceram entre dia 1 e 15:", selecionados.shape)

# Pode usar outros operadores no compare_numeric, confira a documentação para ver as opções
selecionados = idosos[
    compare_numeric(idosos, cod_dia_nascimento, ">", 15) &
    compare_numeric(idosos, cod_dia_nascimento, "<=", 31)
]
print("idosos que nasceram entre dia 16 e 31:", selecionados.shape)

Água e casa: (1706, 1088)
Água e apartamento: (135, 1088)
não aplicavel (água): (33518, 1088)
idosos que nasceram entre dia 1 e 15: (21755, 1088)
idosos que nasceram entre dia 16 e 31: (19407, 1088)
